In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# 1. Simulate wind farm data
np.random.seed(42)

n_samples = 800

# Features
wind_speed = np.random.uniform(2, 20, n_samples)          # m/s
air_density = np.random.normal(1.225, 0.04, n_samples)    # kg/m^3
turbulence = np.random.uniform(0.02, 0.25, n_samples)     # 0–1

# Simple nonlinear power curve (3 MW turbine)
def power_curve(ws):
    p = np.piecewise(
        ws,
        [ws < 3, (ws >= 3) & (ws < 13), (ws >= 13) & (ws < 25), ws >= 25],
        [
            0,
            lambda x: 3_000 * ((x - 3) / (13 - 3))**3,  # cubic ramp
            3_000,
            0,
        ],
    )
    return p

base_power = power_curve(wind_speed)

# Modify by density and turbulence
density_factor = air_density / 1.225
turbulence_factor = 1.0 - 0.5 * turbulence

true_power = base_power * density_factor * turbulence_factor

# Add measurement noise
noise = np.random.normal(0, 120, n_samples)
power_output = np.clip(true_power + noise, 0, 3_000)

X = np.column_stack([wind_speed, air_density, turbulence])
y = power_output

# 2. Train–test split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=0
)

# 3. Scale features (important for KNN)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Fit colorful KNN model

K = 7
knn = KNeighborsRegressor(n_neighbors=K, weights="distance")
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"KNN (K={K}): MAE = {mae:.1f} kW, R^2 = {r2:.3f}")

# 5. Colorful visualization (2D slice: fix density)

# Fix air_density near its mean, vary wind_speed & turbulence on a grid
ws_grid = np.linspace(2, 20, 80)
ti_grid = np.linspace(0.02, 0.25, 80)
WS, TI = np.meshgrid(ws_grid, ti_grid)
rho_fixed = np.full_like(WS, 1.225)

grid_X = np.column_stack([WS.ravel(), rho_fixed.ravel(), TI.ravel()])
grid_X_scaled = scaler.transform(grid_X)
grid_pred = knn.predict(grid_X_scaled).reshape(WS.shape)

plt.figure(figsize=(8, 6))
cmap = plt.get_cmap("viridis")

contour = plt.contourf(
    WS, TI, grid_pred,
    levels=15,
    cmap=cmap
)
cbar = plt.colorbar(contour)
cbar.set_label("Predicted power (kW)", fontsize=11)

# Overlay training points colored by actual power
scatter = plt.scatter(
    X_train[:, 0],          # wind_speed
    X_train[:, 2],          # turbulence
    c=y_train,
    cmap=cmap,
    edgecolor="white",
    linewidth=0.4,
    alpha=0.7,
    s=30,
    label="Training points"
)

plt.xlabel("Wind speed (m/s)", fontsize=11)
plt.ylabel("Turbulence intensity", fontsize=11)
plt.title(f"KNN wind-power map (K={K})", fontsize=13)
plt.legend(loc="upper right", framealpha=0.9)
plt.tight_layout()
plt.show()

# 6. Predict for a new operating condition
# ----------------------------------------
new_point = np.array([[11.5, 1.24, 0.08]])  # ws, density, turbulence
new_point_scaled = scaler.transform(new_point)
new_pred = knn.predict(new_point_scaled)[0]
print(
    f"Predicted power at ws=11.5 m/s, rho=1.24 kg/m^3, TI=0.08: "
    f"{new_pred:.0f} kW"
)